In [1]:
from bs4 import BeautifulSoup
from bs4 import Tag
import re
from string import punctuation, digits
import os
import pymorphy2
import pandas as pd
import numpy as np
from copy import deepcopy
import copy
import pymorphy2
import re

In [2]:
!pip install pymorphy2

    100% |████████████████████████████████| 51kB 3.1MB/s 
    100% |████████████████████████████████| 7.1MB 181kB/s 
  Running setup.py bdist_wheel for docopt ... - \ | done
  Stored in directory: /home/sashamattini/.cache/pip/wheels/9b/04/dd/7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e
Successfully built docopt
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from string import punctuation, digits
punctuation = set(punctuation + '«»—…“”\n\t' + digits)
table = str.maketrans({ch: None for ch in punctuation})

In [3]:
def tokenize(text):
    """
    Функция удаляет все знаки препинания.
    """
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text).replace('\n', '')
    text = re.sub(r'[\s]{2,}', ' ', text)
    return text.split(' ')

In [4]:
def if_exists_filled(new_soup, name, tag, create=False):
    
    a = new_soup.find(name)

    if not a:
        
        if create == True:
            tag.append(new_soup.new_tag(name))
            return 'Created'
        else:
            return None

    elif a.string != 'None':
        return 'Filled'

    else:
        return 'Empty'

In [5]:
def create_style(cur_soup, line, tag, type1='style'):

    usg = cur_soup.new_tag('usg', type=type1)
    usg.string = line
    tag.append(usg)


In [6]:
def create_paradigm(cur_soup, prdgm, line):
    
    if_exists_filled(cur_soup, 'paradigm', cur_soup.gramgrp, create=True)
    
    if prdgm == True:
        cur_soup.paradigm.append(', ' + line)
    else:
        cur_soup.paradigm.append(line)
        prdgm = True
    return prdgm


In [7]:
def create_sense_and_style(cur_soup, sense_n, line, tag, type1='style'):
    sns = cur_soup.new_tag('sense', n=sense_n)
    create_style(cur_soup, line, sns, type1=type1)
    tag.append(sns)

In [8]:
def gram_info(line, cur_soup, style, sense_n, gr2=False):

    # i
    # стиль для прилагательных
    # род существильного
    # род + что-то у существительного
    # число существительного и или + что-то типа падежа

    tokens = re.findall('\[(.)\](.*?)\[/.\]', line)
    prdgm = False
    fill = 0
    
    if tokens:
        for token in tokens:

            if token[0] == 'u':

                prdgm = create_paradigm(cur_soup, prdgm, token[1])
                fill += 1
    
            elif token[0] == 'i':

                words = token[1].split(' ')

                for word in words:

                    if cur_soup.pos.text == 'С':

                        if word in ('ж.', 'м.', 'ср.'):

                            if_exists_filled(cur_soup, 'gender', cur_soup.gramgrp, create=True)
                            cur_soup.gender.string = word
                            fill += 1

                    if word in style:
    
                        if gr2 == True:
                            if if_exists_filled(cur_soup, 'sense', cur_soup.entry) == None:
                                create_sense_and_style(cur_soup, sense_n, word, cur_soup.entry)
                                fill += 1
                                
                        if if_exists_filled(cur_soup, 'usg', cur_soup.entry) != 'Filled':
                            create_style(cur_soup, word, cur_soup.entry)
                            fill += 1

    if cur_soup.pos.text == 'С':
        line = line.strip("-")
        tokens = line.split(',')
        if tokens:
            if '[' not in tokens[0]:
                prdgm = create_paradigm(cur_soup, prdgm, tokens[0])


    if fill == 0 and gr2 == True:
        if if_exists_filled(cur_soup, 'sense', cur_soup.entry) == None:
            smt = re.sub('\[.*?\]', '', line)
            create_sense_and_style(cur_soup, sense_n, smt, cur_soup.entry,  type1='other')
        

In [9]:
def write_sense(cur_soup, sense_n, definition, style):

    Def = if_exists_filled(cur_soup, 'sense', cur_soup.entry)
    
    if Def == 'Empty':
        if sns == 1:
            cur_soup.sense.string = re.sub('\[.*?\]', '', definition)  #definition

    elif Def == None or Def == 'Filled':
        sens = cur_soup.new_tag('sense', n=sns)
        df = cur_soup.new_tag('def')
        txt = cur_soup.new_tag('text')
        txt.string = re.sub('\[.*?\]', '', definition)
        df.append(txt)
        sens.append(df)
        return sens

In [10]:
def write_example(cur_soup, example, senses):

    ct = cur_soup.new_tag('cit')

    for token in re.findall('i\](.+?)\[/i\](.*?)(?:\[|$)', example):

        if token[0] != '':
            qt = cur_soup.new_tag('quote')
            qt.string = token[0]
            ct.append(qt)

        if token[1] != '':
            scr = cur_soup.new_tag('scr', code='')
            scr.string = token[1]
            ct.append(scr)

    senses.append(ct)
    return senses


In [46]:
with open('masR_Ja_parsed.tei', 'w', encoding='utf-8') as g:
    print('ok')

ok


In [47]:
data = pd.read_csv('masR_Ja.txt',  sep='\t', na_values= np.nan)
data = data.fillna(0)

c:\python\python36-32\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
data.head(3)

,0001,mas,Lex,Entr,Lemma,(Gr),(GrAll),(n),(Gr2),Lemma.1,_PoS,_n,МУ,(Def),(Gr3),(Ex),Unnamed: 16
0,2,масР0001A,р,Р,р,0,0,%n=0,0,р,С,%n=0,0.0,[i]см.[/i] [u]эр[/u].,0,0,0.0
1,3,масР0002A,раб,РАБ,раб,"-а', [i]м.[/i]","-а', [i]м.[/i]",%n=1,0,раб,С,1,0.0,"Человек, лишенный средств производства и являю...",0,[i]Рабовладельцы и рабы — первое крупное делен...,0.0
2,4,масР0002B,0,0,раб,0,"-а', [i]м.[/i]",%||,0,раб,С,1,0.0,"Тот, кто является частной собственностью друго...",0,"[i]Три тяжкие доли имела судьба, И первая доля...",0.0


In [49]:
#texts = ['masA-J.txt', 'masK-P.txt', 'masK-P_2.txt', 'masR_Ja.txt']
#for text in texts:
#data = pd.read_csv('masA-J.txt',  sep='\t', na_values= np.nan)
#data = data.fillna(0)

all_ = []
previous = ''
xml_n = '<superEntry><metalemma></metalemma><entry><form><orth></orth><inflection></inflection><gramGrp></gramGrp></form></entry></superEntry>'
sns = 1
style = ['Зоол.', 'Разг.', 'С.-х.', 'зват.', 'Тех.', 'Радио.', 'Рыб.', 'Устар., теперь ирон', 'Архит.', 'Спец.', 'Сад.', 'перен. Устар.', 'повел.', 'Прост. устар.', 'Лит.', 'Типогр.', 'Тех.', 'Воен.', 'Разг. устар.', 'безл., также без доп. Разг.', 'безл. Прост.', 'прост.', 'Лит.', 'Хим.', 'Устар.', 'безл. Разг.', 'перен.; обычно чего.', 'Обл.', 'зоол.', 'разг.', 'архит.', 'хим.', 'безл.', 'устар.', 'обл.', 'шутл.-ирон.', 'перен.', 'Мат.', 'Физ.']
other = ['в знач. сущ.', 'с определением.', 'в знач. союза', 'в знач. прил.', 'в знач. сущ.', 'безл. в знач. сказ.', 'в безл. употр.']


for i in range(data.shape[0]):

    senses = False

    lemma = data.loc[i]['Lemma.1']

    if lemma != previous:

        previous = lemma

        if i != 0:
            all_.append(cur_soup)

        cur_soup = BeautifulSoup(copy.copy(xml_n), 'lxml')
        cur_soup.orth.string = lemma #data.loc[i]['Entr']
        #print(cur_soup)
        sns = 1

    if if_exists_filled(cur_soup, 'pos', cur_soup.gramgrp, create=True) != 'Filled':
        cur_soup.pos.string = str(data.loc[i]['_PoS'])

    for col_name in data.columns[(data.loc[i] != 0) == True].tolist():

        if col_name == 'Entr':
            if_exists_filled(cur_soup, 'stress', cur_soup.entry, create=True)
            cur_soup.stress.string = data.loc[i]['Entr']

        elif col_name == '(Gr)':
            gram_info(data.loc[i]['(Gr)'], cur_soup, style, sns)

        elif col_name == '(Gr2)':
            gram_info(data.loc[i]['(Gr2)'], cur_soup, style, sns, gr2=True)

        elif col_name == '(Def)':
            definition = data.loc[i]['(Def)']
            senses = write_sense(cur_soup, sns, definition, style)
            sns += 1

        elif col_name == '(Ex)':
            if senses:
                example = data.loc[i]['(Ex)']
                senses = write_example(cur_soup, example, senses)

        if senses:
            cur_soup.entry.append(senses)


    if i == data.shape[0] - 1:
        all_.append(cur_soup)


#for i in all_:
    #print(i)
    #print()


print('ЛОШАДКА')

ЛОШАДКА


In [50]:
with open('masR_Ja_parsed.tei', 'a', encoding='utf-8') as f:
    for i in all_:
        i = i.superentry
        f.write(i.prettify() + '\n')
        #print(i)
        #print()

In [53]:
with open('mas.tei', 'w', encoding='utf-8') as f:
    f.write(main_1)

In [52]:
main_1 = '<tei>\n <file_desc>\n  <respstmt>\n   <name>\n    Костяницына Анастасия, Матвеева Александра\n   </name>\
\n  </respstmt>\n  <extent>\n    количество словарных входов: 150 454\n  </extent>\
\n  <sourcedesc>\n   <ref target="ссылка на словарь в интернете">\n     http://feb-web.ru/feb/mas/MAS-abc/default.asp\
\n   </ref>\n   <p>\n     This database was converted from the MAS Dictionary.\n    </p>\n  </sourcedesc>\
\n </file_desc>\n <front>\n  <head>\n   <title dict_id="">\n     Малый Академический Словарь, МАС\n   </title>\
\n   <volume n="" of="">\n   </volume>\n   <author>\n    РАН, Инcтитут лингвистических исследований\n   </author>\
\n   <publisher>\n   </publisher>\n   <editor>\n    А.П. Евгеньева\n   </editor>\n   <edition>\
\n    4-е издание, стереотипное\n   </edition>\n   <translator>\n   </translator>\n   <pubdate>\n    1999\
\n   </pubdate>\n   <isbn>\n   </isbn>\n   <language type="content">\n    <idno type="iso639-3">\n     rus\
\n    </idno>\n   </language>\n   <script type="rus">\n   </script>\n  </head>\n </front>\n <body>\n  <div>'


In [55]:
dict_parts = ['masA-J_parsed.tei', 'masK-P_1_parsed.tei', 'masK-P_2_parsed.tei', 'masR_Ja_parsed.tei']

for part in dict_parts:
    with open('mas.tei', 'a', encoding='utf-8') as f:
        p = open(part, 'r', encoding='utf-8')
        p_text = p.read()
        f.write(p_text)
        p.close()

In [56]:
with open('mas.tei', 'a', encoding='utf-8') as f:
    f.write('  </div>\n </body>\n</tei>')